In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
from epiweeks import Week
import pdb

In [2]:
DIR =  './data-processed/'
models = [file.split(DIR)[1] for file in glob.glob(DIR + '/*') if ".md" not in file]
location_df = pd.read_csv('data-locations/locations.csv')
location_dict = {location_df['location'][i]:location_df['abbreviation'][i] for i in range(len(location_df)) if not pd.isnull(location_df['abbreviation'][i])}
print(models)
print(location_dict)
location_key = location_dict.keys()
# for each model, get all submissions
df_list = []

for model in models:
    model_dir = DIR + '/' + model + '/' 

    all_items_path = np.array(glob.glob(model_dir + '*.csv'))  # list all csv files' paths
    all_items = [path.replace(model_dir, '') for path in all_items_path]  #list of all csv files' names

    """
    remove forecasts that were duplicated in a given week (if any)
    forecasts file should be unique for each epiweek
    """
    subm_dict = {}
    for i, item in enumerate(all_items):
        date = datetime.strptime(item[:10], '%Y-%m-%d')
        epiweek  = date.isocalendar()[1]
        if epiweek in subm_dict.keys():
            if subm_dict[epiweek][0] <= date:
                subm_dict[epiweek] = (date, i)
        else:
            subm_dict[epiweek] = (date, i)

    select = [ value[1] for key, value in subm_dict.items()]
    select_paths = all_items_path[select]


    data_model = []
    for path in select_paths:

        df = pd.read_csv(path)
        
        """
            create epiweek column
        """
        date = path.split('/')[-1][:10]
        # epiweek ends on Saturday, but submission is until Monday. 
        # we can subtract 2 days, thus, submission on Monday will be considered in the prev week  
        # this also aligns submission week and data
        date = datetime.strptime(date, '%Y-%m-%d') - timedelta(days=2)
        forecast_week = Week.fromdate(date)
        df['forecast_week'] = forecast_week
        #pdb.set_trace()
        data_model.append(df)


    # join all dataframes saved in data_model

    """
        select, rename and sort columns
    """
        

    """
        convert location to region abbreviation
    """
    if len(data_model) == 0:
        continue
    df = pd.concat(data_model, ignore_index=True, sort=False)
    df = df.rename(columns={'target': 'ahead'})
    print('before_len', len(df))
    df = df[df['forecast_date'] > '2022-06-05']
    model_list = []
    df['location']= df['location'].astype(str)
    df = df[df['location'].isin(location_dict)]
    df = df[df['ahead'].str.contains("inc death")==True]
    df = df.reset_index()
    print("len:", len(df))
    print(model)
    print(len(data_model))
    if len(df) == 0:
        continue
    print(df)
    for i in range(len(df)): 
        if i % 10000 == 0:
        key = df['location'][i]
        if key not in location_key: 
            continue
        if len(key) == 1: 
            key = '0' + key
        df.at[i, 'location'] = location_dict[key]
        df.at[i, 'ahead'] = df['ahead'][i][0]
        model_list.append(model)
    df['model'] = model
    df = df[['model', 'forecast_week', 'ahead', 'location', 'type', 'quantile', 'value']]
    
    """final_row = {'model': [], 'forecast_week': [], 'ahead':[], 'location':[],'type':[],'quantile':[],
             'value':[]}
    for index, row in df.iterrows():
        if row['quantile'] == 0.5 and model in model_point: 
            final_row['model'].append(row['model'])
            final_row['forecast_week'].append(row['forecast_week'])
            final_row['ahead'].append(row['ahead'])
            final_row['location'].append(row['location'])
            final_row['type'].append('point')
            final_row['quantile'].append(np.nan)
            final_row['value'].append(row['value'])
    df2 = pd.DataFrame(final_row)
    df3 = pd.concat([df,df2], ignore_index = False)"""
    
    df3 = df
    df3 = df3.sort_values(by=['forecast_week', 'location', 'ahead', 'type'], ascending=[True, True,True,True])
    df_list.append(df3)  
    print(len(df))
print("done")

['UCSB-ACTS', 'OliverWyman-Navigator', 'JHUAPL-Bucky', 'example_inverse_cdf-1.png', 'DDS-NBDS', 'BPagano-RtDriven', 'JBUD-HMXK', 'UVA-Ensemble', 'Microsoft-DeepSTIA', 'CU-nochange', 'explore_processed_data.R', 'RobertWalraven-ESG', 'Covid19Sim-Simulator', 'COVIDhub-4_week_ensemble', 'CovidAnalytics-DELPHI', 'JHUAPL-Morris', 'MOBS-GLEAM_COVID', 'UCM_MESALab-FoGSEIR', 'CovidActNow-SEIR_CAN', 'CDDEP-SEIR_MCMC', 'LNQ-ens1', 'SigSci-TS', 'FAIR-NRAR', 'GT-DeepCOVID', 'USF-STPM', 'CU-scenario_mid', 'JHUAPL-Gecko', 'Geneva-DetGrowth', 'UCF-AEM', 'MUNI-VAR', 'USACE-ERDC_SEIR', 'Yu_Group-CLEP', 'NotreDame-FRED', 'UMass-trends_ensemble', 'RPI_UW-Mob_Collision', 'CMU-TimeSeries', 'MUNI-ARIMA', 'UChicago-CovidIL_30_+', 'NotreDame-mobility', 'SDSC_ISG-TrendModel', 'R_forecast_file_validation.Rmd', 'MIT_CritData-GBCF', 'Imperial-ensemble1', 'UT-Osiris', 'Auquan-SEIR', 'STH-3PU', 'LANL-GrowthRate', 'LosAlamos_NAU-CModel_SDVaxVar', 'SWC-TerminusCM', 'Google_Harvard-CPF', 'UChicago-CovidIL_40', 'NCSU-CO

/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6613/310758148.py:38: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6613/310758148.py:38: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6613/310758148.py:38: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6613/310758148.py:38: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6613/310758148.py:38: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=Fa

before_len 14905728
len: 124488
JHUAPL-Bucky
53
           index forecast_date                 ahead target_end_date location  \
0         644336    2022-08-22  1 wk ahead inc death      2022-08-27       US   
1         644339    2022-08-22  1 wk ahead inc death      2022-08-27       US   
2         644341    2022-08-22  1 wk ahead inc death      2022-08-27       US   
3         644344    2022-08-22  1 wk ahead inc death      2022-08-27       US   
4         644346    2022-08-22  1 wk ahead inc death      2022-08-27       US   
...          ...           ...                   ...             ...      ...   
124483  13344804    2022-07-04  7 wk ahead inc death      2022-08-20       78   
124484  13344806    2022-07-04  7 wk ahead inc death      2022-08-20       78   
124485  13344809    2022-07-04  7 wk ahead inc death      2022-08-20       78   
124486  13344811    2022-07-04  7 wk ahead inc death      2022-08-20       78   
124487  13344814    2022-07-04  7 wk ahead inc death      202

KeyboardInterrupt: 

In [7]:
df_final = pd.concat(df_list, ignore_index = True, sort = False)
df_final = df_final.sort_values(by=['model','forecast_week', 'location', 'ahead', 'type'], ascending=[True,True, True,True,True])
df_final.to_csv('./predictions.csv',index=False)
print('done')

done
